In [2]:
# @title Convert NPY to PLY
# @title Step 1: Install necessary libraries
# This cell installs the required Python libraries, 'numpy' for numerical operations
# and 'open3d' for 3D data processing and visualization.
!pip install numpy open3d

# @title Step 2: Import libraries
import numpy as np
import open3d as o3d
from google.colab import files
import matplotlib.pyplot as plt

# @title Step 3: Upload your .npy file
# This cell will prompt you to upload your .npy file from your local machine.
print("Please upload your .npy file.")
uploaded = files.upload()

# Get the filename of the uploaded file
if not uploaded:
    print("No file was uploaded. Please run this cell again to upload the file.")
else:
    npy_filename = list(uploaded.keys())[0]
    print(f"\nSuccessfully uploaded '{npy_filename}'")

    # @title Step 4: Load the depth map and create a point cloud
    # This cell loads the depth data from the .npy file and converts it into an
    # Open3D Image object. It then creates an intrinsic camera matrix, which is
    # necessary to project the 2D depth pixels into 3D space, and finally
    # creates the point cloud from the depth image using these intrinsics.

    try:
        # Load the depth map from the .npy file
        depth_map = np.load(npy_filename)

        print("Depth map loaded successfully. Shape:", depth_map.shape)

        # Convert the NumPy array to an Open3D Image
        # Open3D expects the data type to be uint16 for depth images.
        # We also need to handle the scaling if the depth is not in millimeters.
        # Assuming the depth is in meters, we scale it to millimeters.
        # If your depth is already in millimeters, you can remove the scaling factor of 1000.
        depth_image = o3d.geometry.Image((depth_map).astype(np.uint16))

        # Get the height and width of the depth map
        height, width = depth_map.shape

        # Create an intrinsic camera matrix.
        # You might need to adjust these values based on your camera's properties.
        # fx, fy are focal lengths, cx, cy are principal point offsets.
        fx = 525.0  # example value
        fy = 525.0  # example value
        cx = width / 2
        cy = height / 2
        intrinsics = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

        # Create a point cloud from the depth image and intrinsics
        pcd = o3d.geometry.PointCloud.create_from_depth_image(
            depth_image,
            intrinsics
        )

        # The default creation is upside down, so we flip it
        pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])


        print("\nPoint cloud created successfully.")
        print("Number of points:", len(pcd.points))

        # @title Step 4.5: Save the raw point cloud as a .ply file
        # This cell saves the generated point cloud data to a .ply file before segmentation.
        raw_ply_filename = npy_filename.replace('.npy', '_raw.ply')
        o3d.io.write_point_cloud(raw_ply_filename, pcd)
        print(f"\nRaw point cloud saved as '{raw_ply_filename}'")

        print(f"\nDownloading '{raw_ply_filename}'...")
        files.download(raw_ply_filename)


        # @title Step 5: Segment the point cloud using DBSCAN clustering
        # This cell performs DBSCAN clustering to group points into segments.
        # 'eps' is the distance to neighbors in a cluster.
        # 'min_points' is the minimum number of points required to form a cluster.
        # You can adjust these parameters to change the clustering behavior.
        eps = 0.02  # Tweak this value based on your point cloud density
        min_points = 10 # Tweak this value based on your point cloud density

        labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

        # Get the unique cluster labels
        max_label = labels.max()
        print(f"\nPoint cloud has been clustered into {max_label + 1} segments.")

        # Create a color map for the clusters
        # We use a colormap to assign a unique color to each cluster label.
        # Points labeled as -1 are considered noise and will be colored gray.
        colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
        colors[labels < 0] = 0  # Set noise points to black
        pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

        print("Clusters have been colored.")

        # @title Step 6: Save the segmented point cloud as a .ply file
        # This cell saves the generated and colored point cloud data to a .ply file.
        segmented_ply_filename = npy_filename.replace('.npy', '_segmented.ply')
        o3d.io.write_point_cloud(segmented_ply_filename, pcd)
        print(f"\nSegmented point cloud saved as '{segmented_ply_filename}'")

        # @title Step 7: Download the .ply file
        # This cell will automatically download the generated .ply file to your local machine.
        print(f"\nDownloading '{segmented_ply_filename}'...")
        files.download(segmented_ply_filename)

    except Exception as e:
        print(f"\nAn error occurred: {e}")
        print("Please ensure the uploaded file is a valid .npy file representing a depth map.")

Please upload your .npy file.


Saving manual_depth_20250827-104525.npy to manual_depth_20250827-104525.npy

Successfully uploaded 'manual_depth_20250827-104525.npy'
Depth map loaded successfully. Shape: (480, 640)

Point cloud created successfully.
Number of points: 260516

Raw point cloud saved as 'manual_depth_20250827-104525_raw.ply'



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Point cloud has been clustered into 4 segments.
Clusters have been colored.

Segmented point cloud saved as 'manual_depth_20250827-104525_segmented.ply'



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>